# Medical NER Fine-Tuning with Llama 3.2 3B + LoRA

This notebook implements fine-tuning of Llama 3.2 3B Instruct for medical Named Entity Recognition (NER) using:
- **SFT** (Supervised Fine-Tuning)
- **LoRA** (Low-Rank Adaptation)
- **Hugging Face Hub** integration for checkpoint uploads

## Tasks:
1. Chemical entity extraction
2. Disease entity extraction
3. Chemical-Disease relationship extraction

## Dataset:
- 3,000 medical text examples
- 80/10/10 train/validation/test split
- **⚠️ CRITICAL**: Data is shuffled before splitting to ensure balanced task distribution
- Weights & Biases tracking enabled

## Important Note:
**Data splitting MUST use `shuffle=True`** to prevent task imbalance. Without shuffling, all relationship extraction examples may cluster in validation/test sets, leading to poor model performance on the most important task!

## 0. Environment Variables Setup

⚠️ **IMPORTANT**: Set your credentials before running this notebook!

Required:
- `HF_TOKEN`: Your Hugging Face token (needed to save models to HF Hub)

Optional:
- `WANDB_API_KEY`: Your Weights & Biases API key (for training tracking)

In [ ]:
import os
import getpass

# Secure token input - use environment variables or prompt for input
# Option 1: Set environment variables before running notebook (recommended for automation)
# Option 2: Enter tokens when prompted (recommended for interactive use)

# Hugging Face Token (required for uploading to HF Hub)
hf_token = os.getenv("HF_TOKEN")
if not hf_token or hf_token == "hf_YOUR_TOKEN_HERE":
    hf_token = getpass.getpass("Enter your Hugging Face token: ")
    os.environ["HF_TOKEN"] = hf_token

# Weights & Biases API Key (optional, for training tracking)
wandb_key = os.getenv("WANDB_API_KEY")
if not wandb_key:
    use_wandb = input("Use Weights & Biases for tracking? (y/n): ").lower() == 'y'
    if use_wandb:
        wandb_key = getpass.getpass("Enter your W&B API key (or press Enter to use cached login): ")
        if wandb_key:
            os.environ["WANDB_API_KEY"] = wandb_key

# Verify environment variables
print("✓ Environment variables configured")
print(f"  HF_TOKEN: {'✓ Set' if os.environ.get('HF_TOKEN') else '✗ Not set'}")
print(f"  WANDB_API_KEY: {'✓ Set' if os.environ.get('WANDB_API_KEY') else '○ Will use cached login or prompt later'}")

## 1. Setup and Installation

First, let's install all required dependencies.

In [ ]:
# Install required packages
!pip install -q transformers datasets peft accelerate bitsandbytes
!pip install -q huggingface-hub tokenizers trl scikit-learn
!pip install -q scipy sentencepiece protobuf wandb hf_transfer

print("✓ All packages installed successfully!")

## 2. Import Libraries

In [ ]:
! pip install wandb

In [ ]:
import json
import torch
import os
import random
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from huggingface_hub import login
import wandb

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## 3. Configuration

⚠️ **IMPORTANT**: Update `HF_USERNAME` with your Hugging Face username!

In [ ]:
# Configuration Section
from datetime import datetime

HF_USERNAME = "albyos"  # Replace with your HF username

# Generate timestamp for checkpoint naming
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
HF_MODEL_ID = f"{HF_USERNAME}/llama3-medical-ner-lora-{TIMESTAMP}"
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
MODEL_NAME = BASE_MODEL  # Alias for consistency

# LoRA Configuration
LORA_RANK = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# Training Configuration
NUM_EPOCHS = 5  # Increased from 3 to 5 for better learning of pipe format
BATCH_SIZE = 4
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 5e-5  # Reduced from 2e-4 to prevent over-prediction and improve precision

# Data Configuration
TRAIN_SPLIT_RATIO = 0.9
RANDOM_SEED = 42
RESHUFFLE_SPLITS_EACH_RUN = True  # When True, create a fresh validation split every run
SPLIT_SEED = random.randint(0, 1_000_000) if RESHUFFLE_SPLITS_EACH_RUN else RANDOM_SEED

print("✓ Configuration loaded")
print(f"  Base model: {BASE_MODEL}")
print(f"  HF model ID: {HF_MODEL_ID}")
print(f"  Training timestamp: {TIMESTAMP}")
print(f"  LoRA rank: {LORA_RANK}")
print(f"  Training epochs: {NUM_EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE} (reduced for precision)")
print(f"  Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Data split seed: {SPLIT_SEED} ({'reshuffled' if RESHUFFLE_SPLITS_EACH_RUN else 'fixed'})")

## 0) Reusable Utilities

These utility functions provide text normalization, hashing for deduplication, parsing, and validation for the data processing pipeline.

In [ ]:
# ===== Utilities: normalization, hashing, parsing =====
import re, json, hashlib
from collections import Counter

def dehyphenate(s: str) -> str:
    # Join words broken across lines with hyphens + whitespace
    return re.sub(r"(\w+)-\s+(\w+)", r"\1\2", s)

def normalize_text(s: str) -> str:
    s = dehyphenate(s or "")
    s = s.lower()
    s = re.sub(r"[\u00A0\t\r\n]+", " ", s)     # spaces/newlines
    s = re.sub(r"\s+", " ", s).strip()
    return s

def prompt_hash(prompt: str) -> str:
    return hashlib.md5(normalize_text(prompt).encode("utf-8")).hexdigest()

def parse_bullets(text: str):
    items = []
    for line in (text or "").splitlines():
        m = re.match(r"^\s*[-*]\s*(.+?)\s*$", line)
        if m:
            items.append(m.group(1))
    return items

def normalize_item(s: str) -> str:
    s = (s or "").lower()
    # Keep hyphens intact (e.g., "type-2 diabetes" stays "type-2 diabetes")
    s = re.sub(r"\s+", " ", s)  # Only normalize whitespace
    s = re.sub(r"[\.,;:]+$", "", s).strip()
    return s

def in_text(item: str, text: str) -> bool:
    return normalize_item(item) in normalize_text(text)

def unique_preserve_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x); out.append(x)
    return out

print("✓ Utility functions loaded")

## 4. Hugging Face Authentication

Get your token from: https://huggingface.co/settings/tokens

In [ ]:
# Authenticate with Hugging Face
from huggingface_hub import login

hf_token = os.environ.get("HF_TOKEN")
if hf_token and hf_token != "hf_YOUR_TOKEN_HERE":
    login(token=hf_token)
    print("✓ Logged in to Hugging Face")
else:
    print("⚠ HF_TOKEN not set. Please update Cell 3 before continuing.")

## 4b. Weights & Biases Setup

Initialize W&B to track training metrics, validation loss, and experiments.
Get your API key from: https://wandb.ai/authorize

In [ ]:
# Login to Weights & Biases
wandb_key = os.getenv('WANDB_API_KEY')

if wandb_key and wandb_key != 'your_wandb_key_here':
    wandb.login(key=wandb_key)
    print('✓ Logged in to Weights & Biases using WANDB_API_KEY')
else:
    print('⚠ Warning: WANDB_API_KEY not set. Attempting to use cached login...')
    try:
        wandb.login()
        print('✓ Logged in to Weights & Biases using cached credentials')
    except Exception as e:
        print(f'⚠ Warning: Could not login to W&B: {e}')
        print('  Run wandb.login() interactively or set WANDB_API_KEY environment variable')

In [ ]:
# Initialize Weights & Biases
wandb.init(
    project="medical-ner-finetuning",
    name=f"llama3-medical-ner-{TIMESTAMP}",
    config={
        "model": BASE_MODEL,
        "lora_rank": LORA_RANK,
        "lora_alpha": LORA_ALPHA,
        "learning_rate": LEARNING_RATE,
        "epochs": NUM_EPOCHS,
        "batch_size": BATCH_SIZE * GRADIENT_ACCUMULATION,
    }
)

print("✓ Weights & Biases initialized")
print(f"  Project: medical-ner-finetuning")
print(f"  Run name: llama3-medical-ner-{TIMESTAMP}")
print(f"  Dashboard: https://wandb.ai")

## 5. Data Exploration

Let's examine the dataset structure.

In [ ]:
# Load and inspect the dataset
# Load data
with open('both_rel_instruct_all.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

print(f"Total samples: {len(data)}")
print(f"\nSample structure:")
print(json.dumps(data[0], indent=2)[:500] + "...")

In [ ]:
# Analyze task distribution
# IMPORTANT: Check for "influences between" FIRST because those prompts also contain "chemicals" and "diseases"
task_counts = {}
for sample in data:
    if "influences between" in sample['prompt']:
        task = "Relationship Extraction"
    elif "chemicals mentioned" in sample['prompt']:
        task = "Chemical Extraction"
    elif "diseases mentioned" in sample['prompt']:
        task = "Disease Extraction"
    else:
        task = "Other"
    
    task_counts[task] = task_counts.get(task, 0) + 1

print("Task Distribution:")
for task, count in task_counts.items():
    print(f"  {task}: {count} ({count/len(data)*100:.1f}%)")

In [ ]:
# Show example from each task type
print("="*80)
print("EXAMPLE: Chemical Extraction")
print("="*80)
chem_example = [s for s in data if "chemicals mentioned" in s['prompt'] and "influences between" not in s['prompt']][0]
print(f"Prompt:\n{chem_example['prompt'][:300]}...")
print(f"\nCompletion:\n{chem_example['completion']}")

print("\n" + "="*80)
print("EXAMPLE: Disease Extraction")
print("="*80)
disease_example = [s for s in data if "diseases mentioned" in s['prompt'] and "influences between" not in s['prompt']][0]
print(f"Prompt:\n{disease_example['prompt'][:300]}...")
print(f"\nCompletion:\n{disease_example['completion']}")

print("\n" + "="*80)
print("EXAMPLE: Relationship Extraction")
print("="*80)
rel_example = [s for s in data if "influences between" in s['prompt']][0]
print(f"Prompt:\n{rel_example['prompt'][:300]}...")
print(f"\nCompletion:\n{rel_example['completion']}")


## 6. Leak-Free Stratified Dataset Splitting

⚠️ **CRITICAL**: Using **leak-free stratified splitting** with deduplication to prevent data contamination!

**Key Improvements**:
1. **Deduplication**: Remove duplicate prompts (by normalized hash) before splitting
2. **Stratified Splitting**: Ensures exact task distribution across all splits
3. **Leakage Prevention**: Hard assertions verify zero overlap between train/val/test

Split into:
- **80% Training** - for fine-tuning
- **10% Validation** - for monitoring during training (W&B)
- **10% Test** - for final evaluation after training

**Guaranteed distribution in each split** (with stratification):
- **Exactly 33.3%** Chemical extraction
- **Exactly 33.3%** Disease extraction  
- **Exactly 33.3%** Relationship extraction

**Why this matters**:
- Duplicates can leak between splits, causing overfitting
- Normalized prompts ensure semantic uniqueness
- Hard assertions catch any data leakage bugs

In [ ]:
# ===== 1) Deduplicate + Stratified Split (80/10/10) =====
from sklearn.model_selection import train_test_split
import random, os, time

random.seed(SPLIT_SEED)

# 1.1 Load full dataset into `data` (already loaded above)
assert isinstance(data, list) and len(data) > 0, "Load `data` first"

# 1.1.5 Define task classifier (needed for format conversion)
def task_from_prompt(prompt: str) -> str:
    p = normalize_text(prompt)
    if "list of extracted chemicals" in p: return "chemicals"
    if "list of extracted diseases"  in p: return "diseases"
    if "list of extracted influences" in p: return "influences"
    # Fallback patterns
    if "chemicals mentioned" in p: return "chemicals"
    if "diseases mentioned" in p: return "diseases"
    if "influences between" in p: return "influences"
    return "other"

# 1.2 Deduplicate by normalized prompt
seen = set()
clean = []
duplicates_removed = 0
for row in data:
    ph = prompt_hash(row["prompt"])
    if ph in seen: 
        duplicates_removed += 1
        continue
    seen.add(ph)
    
    # Dedupe and normalize completion lines
    items = unique_preserve_order(parse_bullets(row.get("completion","")))
    items_norm = []
    
    # Convert OLD sentence format to NEW pipe format for influences
    task = task_from_prompt(row["prompt"])
    for item in items:
        if not item.strip():
            continue
        
        if task == "influences":
            # Convert: "chemical X influences disease Y" → "X | Y"
            m = re.match(r'^\s*chemical\s+(.+?)\s+influences\s+disease\s+(.+?)\s*$', item, re.I)
            if m:
                chem = normalize_item(m.group(1))
                dis = normalize_item(m.group(2))
                items_norm.append(f"{chem} | {dis}")
            else:
                # Already in pipe format or other format - keep as-is
                items_norm.append(normalize_item(item))
        else:
            # Chemicals or diseases - just normalize
            items_norm.append(normalize_item(item))
    
    items_norm = unique_preserve_order(items_norm)
    row["completion"] = "\n".join(f"- {x}" for x in items_norm)
    clean.append(row)

data = clean
print(f"✓ Deduplication complete:")
print(f"  Original samples: {len(data) + duplicates_removed}")
print(f"  Duplicates removed: {duplicates_removed}")
print(f"  Unique samples: {len(data)}")
print(f"  ✓ Influences converted to pipe format (chemical | disease)")

# 1.3 Get task labels for stratification
labels = [task_from_prompt(r["prompt"]) for r in data]
print(f"\n✓ Task label distribution: {dict(Counter(labels))}")

# 1.4 First split: 80% train, 20% temp
train_data, temp_data, train_y, temp_y = train_test_split(
    data, labels, test_size=0.2, random_state=SPLIT_SEED, stratify=labels
)

# 1.5 Second split: 10% val, 10% test (split temp 50/50)
val_data, test_data, val_y, test_y = train_test_split(
    temp_data, temp_y, test_size=0.5, random_state=SPLIT_SEED+1, stratify=temp_y
)

# 1.6 Check for leakage
def check_leak(a, b, name):
    ha = {prompt_hash(r["prompt"]) for r in a}
    hb = {prompt_hash(r["prompt"]) for r in b}
    overlap = ha & hb
    print(f"  {name}: overlap={len(overlap)}")
    assert len(overlap) == 0, f"❌ Leak detected between {name}"

print(f"\n✓ Checking for data leakage...")
check_leak(train_data, val_data, "train ∩ val")
check_leak(train_data, test_data, "train ∩ test")
check_leak(val_data,   test_data, "val ∩ test")
print(f"✓ No leakage detected - all splits are independent!")

# 1.7 Write files
out_train = "train.jsonl"
out_val   = "validation.jsonl"
out_test  = "test.jsonl"

for path, split in [(out_train,train_data),(out_val,val_data),(out_test,test_data)]:
    with open(path,"w",encoding="utf-8") as f:
        for r in split:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"\n✓ Dataset split complete (seed={SPLIT_SEED}, stratified=True, deduplicated=True)")
print(f"  Train: {len(train_data)} samples ({len(train_data)/len(data)*100:.1f}%)")
print(f"  Validation: {len(val_data)} samples ({len(val_data)/len(data)*100:.1f}%)")
print(f"  Test: {len(test_data)} samples ({len(test_data)/len(data)*100:.1f}%)")
print(f"\n  Files written: {out_train}, {out_val}, {out_test}")

## 🔧 Critical Format Conversion Applied

**Problem Identified:**  
The source data uses OLD format for influences:
```
"- chemical cyclophosphamide influences disease urinary bladder cancer"
```

But for better model performance, we want NEW format:
```
"- cyclophosphamide | urinary bladder cancer"
```

**Solution:**  
During deduplication, we now automatically convert influences from sentence format to pipe format. This ensures:
1. Model learns concise `"chemical | disease"` format
2. Cleaner outputs with less verbosity
3. Easier to parse during evaluation
4. Consistent format across all relationship extractions

The conversion happens in the deduplication loop below.

In [ ]:
# Verify task distribution across splits
def get_task_type_display(prompt):
    """Classify the task type based on prompt for display."""
    task = task_from_prompt(prompt)
    display_map = {
        "chemicals": "Chemical Extraction",
        "diseases": "Disease Extraction", 
        "influences": "Relationship Extraction",
        "other": "Other"
    }
    return display_map.get(task, "Other")

def verify_split_distribution(split_data, split_name):
    """Verify task distribution in a split."""
    task_counts = {}
    for sample in split_data:
        task = get_task_type_display(sample['prompt'])
        task_counts[task] = task_counts.get(task, 0) + 1
    
    print(f"\n{split_name} Distribution ({len(split_data)} samples):")
    for task, count in sorted(task_counts.items()):
        print(f"  {task}: {count} ({count/len(split_data)*100:.1f}%)")

verify_split_distribution(train_data, "Train")
verify_split_distribution(val_data, "Validation")
verify_split_distribution(test_data, "Test")

print("\n✓ All splits have balanced task distributions")

## 6b. Post-Split Leakage Verification

Double-check that written files have no overlapping prompts.

In [ ]:
# ===== Post-split leakage check (standalone verification) =====
def hash_file(path):
    """Load all prompt hashes from a JSONL file."""
    out=set()
    with open(path,"r",encoding="utf-8") as f:
        for line in f:
            r=json.loads(line)
            out.add(prompt_hash(r["prompt"]))
    return out

Htrain = hash_file("train.jsonl")
Hval   = hash_file("validation.jsonl")
Htest  = hash_file("test.jsonl")

print("✓ Verifying written files for leakage...")
print(f"  train ∩ val: {len(Htrain & Hval)} overlaps")
print(f"  train ∩ test: {len(Htrain & Htest)} overlaps")
print(f"  val ∩ test: {len(Hval & Htest)} overlaps")

assert not (Htrain & Hval),   "❌ Leak detected: train ∩ val"
assert not (Htrain & Htest),  "❌ Leak detected: train ∩ test"
assert not (Hval & Htest),    "❌ Leak detected: val ∩ test"

print("\n✓✓✓ No cross-split leakage in written files!")

## 7. Data Formatting

Format data into Llama 3 chat format with system, user, and assistant roles.

In [ ]:
def format_instruction(sample):
    """Format data into Llama 3 chat format with strict instruction."""
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Return ONLY entities that appear verbatim in the article.
Output one item per line, each starting with '- '.
For relationships: use format '- chemical NAME | disease NAME'.
For single entities: use format '- ENTITY NAME'.
If none exist, return nothing.
Do not add explanations or examples.<|eot_id|><|start_header_id|>user<|end_header_id|>

{sample['prompt']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{sample['completion']}<|eot_id|>"""

# Test formatting
formatted_example = format_instruction(train_data[0])
print("Formatted Example:")
print(formatted_example[:500] + "...")

In [ ]:
# Format all data
train_formatted = [{"text": format_instruction(sample)} for sample in train_data]
val_formatted = [{"text": format_instruction(sample)} for sample in val_data]
test_formatted = [{"text": format_instruction(sample)} for sample in test_data]

# Create HuggingFace datasets
train_dataset = Dataset.from_list(train_formatted)
val_dataset = Dataset.from_list(val_formatted)
test_dataset = Dataset.from_list(test_formatted)

print(f"✓ Datasets formatted:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Validation: {len(val_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

## 8. Load Model and Tokenizer

Load Llama 3.2 3B with 4-bit quantization for memory efficiency.

In [ ]:
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("✓ Quantization config created (4-bit NF4)")

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    padding_side="right",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

print(f"✓ Tokenizer loaded: {MODEL_NAME}")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  PAD token: {tokenizer.pad_token}")
print(f"  EOS token: {tokenizer.eos_token}")

In [ ]:
# Load base model
print("Loading model... (this may take a few minutes)")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print(f"✓ Base model loaded: {MODEL_NAME}")
print(f"  Model size: {model.get_memory_footprint() / 1e9:.2f} GB")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
print("✓ Model prepared for k-bit training")

## 9. Configure LoRA

Apply Low-Rank Adaptation for efficient fine-tuning.

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=LORA_RANK,                   # LoRA rank
    lora_alpha=LORA_ALPHA,         # LoRA alpha (scaling)
    target_modules=[               # Layers to apply LoRA
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,             # Dropout for regularization
    bias="none",                   # No bias training
    task_type="CAUSAL_LM"          # Causal language modeling
)

print(f"✓ LoRA configuration:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Target modules: {len(lora_config.target_modules)}")

In [ ]:
# Apply LoRA to model
model = get_peft_model(model, lora_config)

print("✓ LoRA applied to model")
print("\nTrainable parameters:")
model.print_trainable_parameters()

## 10. Tokenize Datasets

In [ ]:
def tokenize_function(examples):
    """Tokenize the texts."""
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )

# Tokenize datasets
print("Tokenizing datasets...")

tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train set"
)

tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names,
    desc="Tokenizing validation set"
)

print(f"✓ Train set tokenized: {len(tokenized_train)} samples")
print(f"✓ Validation set tokenized: {len(tokenized_val)} samples")

In [ ]:
# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked LM
)

print("✓ Data collator created")

## 11. Training Configuration

In [ ]:
# Training arguments
training_args = TrainingArguments(
    # Output and logging
    output_dir="./llama3-medical-ner-lora",
    logging_dir="./logs",
    logging_steps=10,
    
    # Training parameters
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    
    # Optimization
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=50,  # Evaluate every 50 steps
    
    # Checkpointing - Save every 50 steps
    save_strategy="steps",
    save_steps=50,  # Checkpoint every 50 steps
    save_total_limit=None,  # Keep all checkpoints (will push to HF with unique names)
    
    # Memory optimization
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    
    # Mixed precision
    fp16=True,
    
    # Hugging Face Hub - Disable default push (we'll use custom callback)
    push_to_hub=False,  # Custom callback will handle timestamped uploads
    
    # Misc
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="wandb",  # Enable Weights & Biases logging
    run_name=f"llama3-medical-ner-{TIMESTAMP}",  # W&B run name
    seed=42,
)

print(f"✓ Training configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size (per device): {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Checkpoint frequency: Every {training_args.save_steps} steps")
print(f"  Base HF model ID: {HF_MODEL_ID}")
print(f"  ⚠️ Checkpoints will be pushed to HF with timestamp suffix")

## 11b. Custom Checkpoint Upload Callback

This callback will automatically push each checkpoint to Hugging Face Hub with a unique timestamped name every 50 steps.

In [ ]:
from transformers import TrainerCallback
from huggingface_hub import HfApi
import shutil
from pathlib import Path

class CheckpointUploadCallback(TrainerCallback):
    """
    Custom callback to upload checkpoints to Hugging Face Hub with timestamped names.
    Deletes local checkpoints after successful upload to save disk space.
    
    Each checkpoint will be saved with format:
    {HF_USERNAME}/llama3-medical-ner-checkpoint-{step}-{timestamp}
    """
    
    def __init__(self, base_model_id, hf_username, delete_after_upload=True):
        self.base_model_id = base_model_id
        self.hf_username = hf_username
        self.delete_after_upload = delete_after_upload
        self.api = HfApi()
        
    def on_save(self, args, state, control, **kwargs):
        """
        Called when a checkpoint is saved.
        Uploads the checkpoint to HF Hub with a timestamped name, then deletes local copy.
        """
        # Get the checkpoint directory that was just saved
        checkpoint_dir = f"{args.output_dir}/checkpoint-{state.global_step}"
        
        if not Path(checkpoint_dir).exists():
            print(f"⚠️ Checkpoint directory not found: {checkpoint_dir}")
            return
        
        # Create timestamped model ID
        checkpoint_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint_model_id = f"{self.hf_username}/llama3-medical-ner-checkpoint-{state.global_step}-{checkpoint_timestamp}"
        
        print(f"\n{'='*80}")
        print(f"📤 Uploading checkpoint to Hugging Face Hub")
        print(f"   Step: {state.global_step}")
        print(f"   Model ID: {checkpoint_model_id}")
        print(f"{'='*80}\n")
        
        try:
            # Create repository if it doesn't exist
            try:
                self.api.create_repo(
                    repo_id=checkpoint_model_id,
                    repo_type="model",
                    exist_ok=True,
                    private=False
                )
                print(f"✓ Repository created/verified: {checkpoint_model_id}")
            except Exception as create_error:
                print(f"⚠️  Repository creation note: {create_error}")
            
            # Upload the checkpoint folder to HF Hub
            self.api.upload_folder(
                folder_path=checkpoint_dir,
                repo_id=checkpoint_model_id,
                repo_type="model",
                commit_message=f"Checkpoint at step {state.global_step}",
            )
            
            print(f"✅ Checkpoint uploaded successfully!")
            print(f"   URL: https://huggingface.co/{checkpoint_model_id}")
            
            # Delete local checkpoint after successful upload
            if self.delete_after_upload:
                try:
                    shutil.rmtree(checkpoint_dir)
                    print(f"🗑️  Local checkpoint deleted to save disk space")
                except Exception as delete_error:
                    print(f"⚠️  Could not delete local checkpoint: {delete_error}")
            
            print()  # Empty line for readability
            
            # Log to wandb if available
            if wandb.run is not None:
                wandb.log({
                    "checkpoint_step": state.global_step,
                    "checkpoint_url": f"https://huggingface.co/{checkpoint_model_id}"
                })
                
        except Exception as e:
            print(f"❌ Failed to upload checkpoint: {e}")
            print(f"   Checkpoint saved locally at: {checkpoint_dir}\n")

# Initialize the callback with automatic deletion after upload
checkpoint_upload_callback = CheckpointUploadCallback(
    base_model_id=HF_MODEL_ID,
    hf_username=HF_USERNAME,
    delete_after_upload=True  # Delete local checkpoints after upload
)

print(f"✓ Checkpoint upload callback initialized")
print(f"  Checkpoints will be uploaded to: {HF_USERNAME}/llama3-medical-ner-checkpoint-<step>-<timestamp>")
print(f"  Upload frequency: Every {training_args.save_steps} steps")
print(f"  Local deletion: Enabled (checkpoints deleted after upload)")
print(f"  Auto-create repos: Enabled (repos will be created automatically)")


In [ ]:
# Preview expected checkpoint uploads
total_steps_estimate = (len(train_data) // (BATCH_SIZE * GRADIENT_ACCUMULATION)) * NUM_EPOCHS
checkpoint_count = total_steps_estimate // 50

print("="*80)
print("CHECKPOINT UPLOAD PREVIEW")
print("="*80)
print(f"\nTraining Configuration:")
print(f"  Total samples: {len(train_data)}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"  Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Estimated total steps: ~{total_steps_estimate}")
print(f"\nCheckpoint Configuration:")
print(f"  Frequency: Every 50 steps")
print(f"  Expected checkpoints: ~{checkpoint_count}")
print(f"  Local storage: ./llama3-medical-ner-lora/checkpoint-<step>/")
print(f"\nHugging Face Upload:")
print(f"  Format: {HF_USERNAME}/llama3-medical-ner-checkpoint-<step>-<timestamp>")
print(f"\nExample checkpoint names:")
for i, step in enumerate(range(50, min(250, total_steps_estimate), 50), 1):
    example_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(f"  {i}. {HF_USERNAME}/llama3-medical-ner-checkpoint-{step}-{example_time}")
if checkpoint_count > 4:
    print(f"  ... (~{checkpoint_count - 4} more checkpoints)")
    
print(f"\nFinal model:")
print(f"  {HF_USERNAME}/llama3-medical-ner-lora-final-<timestamp>")
print("="*80)

## 12. Initialize Trainer

In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    callbacks=[checkpoint_upload_callback],  # Add custom checkpoint upload callback
)

# Configure early stopping to prevent overfitting
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.0))

# Calculate training steps
total_steps = (len(tokenized_train) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)) * training_args.num_train_epochs

print(f"✓ Trainer initialized")
print(f"✓ Expected training steps: ~{total_steps}")
print(f"✓ Expected checkpoints: ~{max(1, total_steps // training_args.save_steps)}")
print(f"✓ Checkpoint upload callback enabled")
print("✓ Early stopping enabled (patience = 3 evaluations)")
print(f"\n📋 Checkpoint naming format:")
print(f"   {HF_USERNAME}/llama3-medical-ner-checkpoint-<step>-<timestamp>")
print(f"\n   Example: {HF_USERNAME}/llama3-medical-ner-checkpoint-50-20251104_143022")

## 13. Start Training

⚠️ **This will take 2-3 hours on an A100 GPU**

The training will:
- **Save checkpoints every 50 steps** to local disk
- **Upload each checkpoint to Hugging Face Hub** with timestamped names
  - Format: `{username}/llama3-medical-ner-checkpoint-{step}-{timestamp}`
  - Example: `albyos/llama3-medical-ner-checkpoint-50-20251104_143022`
- Evaluate on validation set every 50 steps
- Save the best model based on validation loss
- Log all metrics to Weights & Biases

**Checkpoint URLs will be printed during training and logged to W&B.**

In [ ]:
# Start training
print("="*80)
print("STARTING TRAINING")
print("="*80)
print("This may take 2-3 hours on A100 GPU...\n")

trainer.train()

## 14. Save Final Model

In [ ]:
# Save model locally
print("Saving final model...")
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

print(f"✓ Model saved to: ./final_model")

In [ ]:
# Push final model to Hugging Face Hub with timestamped name
print("Pushing final model to Hugging Face Hub...")

# Create final model ID with timestamp
final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_id = f"{HF_USERNAME}/llama3-medical-ner-lora-final-{final_timestamp}"

try:
    # Push the final model
    model.push_to_hub(
        final_model_id,
        commit_message="Training complete - final model"
    )
    tokenizer.push_to_hub(
        final_model_id,
        commit_message="Training complete - final tokenizer"
    )
    
    print(f"✅ Final model pushed successfully!")
    print(f"   Model ID: {final_model_id}")
    print(f"   URL: https://huggingface.co/{final_model_id}")
    
    # Log to wandb
    if wandb.run is not None:
        wandb.log({
            "final_model_url": f"https://huggingface.co/{final_model_id}",
            "final_model_id": final_model_id
        })
        
except Exception as e:
    print(f"⚠ Failed to push to hub: {e}")
    print("  Final model saved locally at: ./final_model")
    print(f"  You can manually push later using:")
    print(f"    model.push_to_hub('{final_model_id}')")

## 15. Training Analysis

In [ ]:
# Plot training metrics
import pandas as pd
import matplotlib.pyplot as plt

# Get training history
log_history = trainer.state.log_history

# Extract losses
train_loss = [entry['loss'] for entry in log_history if 'loss' in entry]
eval_loss = [entry['eval_loss'] for entry in log_history if 'eval_loss' in entry]

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss', color='blue')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(eval_loss, label='Validation Loss', color='orange')
plt.xlabel('Evaluation Steps')
plt.ylabel('Loss')
plt.title('Validation Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Training metrics plotted and saved to: training_metrics.png")

In [ ]:
# Summary statistics
print("="*80)
print("TRAINING SUMMARY")
print("="*80)
print(f"Total training steps: {len(train_loss)}")
print(f"Final training loss: {train_loss[-1]:.4f}")
print(f"Final validation loss: {eval_loss[-1]:.4f}")
print(f"Best validation loss: {min(eval_loss):.4f}")
print(f"Loss reduction: {((train_loss[0] - train_loss[-1]) / train_loss[0] * 100):.1f}%")

## Next Steps

Training is complete! Your model has been saved.

**To evaluate your model:**
1. Open `Medical_NER_Evaluation.ipynb`
2. Run the evaluation on the test set
3. Test custom examples

**Model locations:**
- Local: `./final_model`
- HuggingFace Hub: Check the output above for your model URL
